In [3]:
import os
import re
import csv
import numpy as np
import matplotlib.pyplot as plt

# ===============================================
# MODULE: DAMAGE LEVELS (time-resolved calculation)
# ===============================================

eq_data = [
    (3600,    4e6),    # 1 hour
    (43200,   2e7),    # 12 hours
    (86400,   2.4e7),  # 1 day
    (864000,  4e7),    # 10 days
    (2592000, 5.5e7),  # 1 month
    (15552000,1.1e8),  # 6 months
    (31536000,1.5e8)   # 1 year
]

def level_from_ratio(r):
    if r < 1:
        return 1
    elif r < 4:
        return 2
    elif r < 7:
        return 3
    elif r < 10:
        return 4
    else:
        return 5

def calculate_damage_level(acc_dose, EQ, current_level):
    ratio = acc_dose / EQ if EQ > 0 else float('inf')
    lvl = level_from_ratio(ratio)
    return lvl, ratio, lvl

# -----------------------------------------------
# Helpers para autodetectar batches y time_steps
# -----------------------------------------------
def find_batches(base_dir="."):
    """Devuelve la lista ordenada de números de batch detectados en carpetas output_batch_N."""
    batches = []
    for name in os.listdir(base_dir):
        m = re.match(r"output_batch_(\d+)$", name)
        if m and os.path.isdir(os.path.join(base_dir, name)):
            batches.append(int(m.group(1)))
    return sorted(batches)

def load_time_steps_for_batch(batch_num, dose_methods):
    """
    Intenta cargar el vector de time_steps (en segundos) para un batch:
    1) Si existe un 'total_accumulated_*.csv', lee su primera columna.
    2) Si no, toma cualquier 'dose_rate_..._class_1.csv' y lee la primera columna.
    """
    plots_dir = os.path.join(f"output_batch_{batch_num}", "dose_rate_plots")

    # 1) ¿Hay algún total acumulado ya generado?
    for label in dose_methods.keys():
        total_csv = os.path.join(plots_dir, f"total_accumulated_{label.replace(' ', '_')}.csv")
        if os.path.exists(total_csv):
            # primera columna = tiempo (s)
            try:
                data = np.loadtxt(total_csv, delimiter=',', skiprows=1, usecols=0)
                return np.asarray(data, dtype=float)
            except Exception:
                pass

    # 2) Buscar un CSV de una clase para deducir el eje temporal
    candidates = []
    for prefix in dose_methods.values():
        candidates.append(os.path.join(plots_dir, f"{prefix}1.csv"))
    for cand in candidates:
        if os.path.exists(cand):
            try:
                # asume formato: [Time(s), class1, class2, ...] -> cogemos la 1ª columna
                arr = np.loadtxt(cand, delimiter=',', skiprows=1)
                return np.asarray(arr[:, 0], dtype=float)
            except Exception:
                continue

    raise FileNotFoundError(
        f"No pude deducir 'time_steps' para el batch {batch_num}. "
        f"Revisa que existan CSVs en {plots_dir}."
    )

# ===============================================
# MAIN LOOP
# ===============================================

dose_methods = {
    "Point Source":      "dose_rate_point_source_class_",
    "With Shield":       "dose_rate_shield_class_",
    "Sphere":            "dose_rate_sphere_class_",
    "Circular Base":     "dose_rate_circular_base_class_",
}

batch_list = find_batches(".")
if not batch_list:
    raise RuntimeError("No se encontraron carpetas 'output_batch_N'. Crea/coloca los datos primero.")

for batch_num in batch_list:
    print(f"\n=== Batch {batch_num} — Damage Levels (time-resolved) ===")

    # Cargar/descubrir time_steps para este batch
    time_steps = load_time_steps_for_batch(batch_num, dose_methods)
    if time_steps.ndim != 1:
        time_steps = np.ravel(time_steps)
    if not np.all(np.diff(time_steps) >= 0):
        # Garantizar orden creciente (por si acaso)
        idx = np.argsort(time_steps)
        time_steps = time_steps[idx]

    out_levels_dir = f"damage_levels_{batch_num}"
    csv_levels_dir = os.path.join(out_levels_dir, "csv_data")
    os.makedirs(csv_levels_dir, exist_ok=True)

    max_t = float(time_steps.max())
    segments = [(t, EQ) for t, EQ in eq_data if t <= max_t]
    if not segments:
        # Si todas las EQ quedan por encima de max_t, al menos usa el primer tramo.
        segments = [eq_data[0]]

    accumulated = {}

    # ===============================================
    # LOAD OR GENERATE TOTAL ACCUMULATED DOSES
    # ===============================================
    plots_dir = os.path.join(f"output_batch_{batch_num}", "dose_rate_plots")

    for label, prefix in dose_methods.items():
        total_csv = os.path.join(plots_dir, f"total_accumulated_{label.replace(' ', '_')}.csv")

        if os.path.exists(total_csv):
            # solo la columna de dosis acumulada
            data = np.loadtxt(total_csv, delimiter=',', skiprows=1, usecols=1)
            # si el fichero trae tiempos distintos, re-usa los 'time_steps' ya cargados
            accumulated[label] = np.asarray(data, dtype=float)
        else:
            # construir total acumulado sumando clases 1..9
            cum_sum = np.zeros(len(time_steps), dtype=float)
            for class_num in range(1, 10):
                file_i = os.path.join(plots_dir, f"{prefix}{class_num}.csv")
                if not os.path.exists(file_i):
                    raise FileNotFoundError(f"Missing {file_i}")

                arr = np.loadtxt(file_i, delimiter=',', skiprows=1)
                # Verificar que el eje temporal coincide; si no, interpolar
                t_i = arr[:, 0].astype(float)
                vals = arr[:, 1:].astype(float)  # columnas de tasas por isótopos de la clase
                # Convertir a dosis por paso en rads: suma de columnas y dividir entre 3600 (si eran rads/h)
                step = vals.sum(axis=1) / 3600.0

                if len(t_i) != len(time_steps) or np.max(np.abs(t_i - time_steps)) > 1e-6:
                    # Interpolar sobre el eje común 'time_steps'
                    step = np.interp(time_steps, t_i, step)

                cum = np.cumsum(step)
                cum_sum += cum

            # Guardar CSV total
            with open(total_csv, 'w', newline='') as f:
                w = csv.writer(f)
                w.writerow(["Time (s)", "Accumulated_Dose (rads)"])
                for t, val in zip(time_steps, cum_sum):
                    w.writerow([float(t), float(val)])

            accumulated[label] = cum_sum

    # ===============================================
    # CALCULATE TIME-RESOLVED DAMAGE LEVELS
    # ===============================================
    for label, dose_arr in accumulated.items():
        key = label.replace(' ', '_').lower()
        csv_out = os.path.join(csv_levels_dir, f"{key}_dose_data.csv")
        damage_csv_out = os.path.join(csv_levels_dir, f"{key}_damage_levels.csv")
        png_out = os.path.join(out_levels_dir, f"damage_level_{key}.png")

        # --- Save accumulated dose data if not existing ---
        if not os.path.exists(csv_out):
            with open(csv_out, 'w', newline='') as f:
                w = csv.writer(f)
                w.writerow(["Time Step (s)", "Accumulated Dose (rads)"])
                for t, val in zip(time_steps, dose_arr):
                    w.writerow([float(t), float(val)])

        # --- Initialize arrays ---
        damage_levels_over_time = np.zeros(len(time_steps), dtype=int)
        ratios_over_time = np.zeros(len(time_steps), dtype=float)
        EQ_used_over_time = np.zeros(len(time_steps), dtype=float)

        current_level = 1
        segment_idx = 0

        # --- Compute for each timestep ---
        for i, t in enumerate(time_steps):
            while segment_idx + 1 < len(segments) and t >= segments[segment_idx + 1][0]:
                segment_idx += 1

            EQ = segments[segment_idx][1]
            EQ_used_over_time[i] = EQ
            acc_dose = dose_arr[i]

            level, ratio, _ = calculate_damage_level(acc_dose, EQ, current_level)
            current_level = level
            damage_levels_over_time[i] = level
            ratios_over_time[i] = ratio

        # --- Save detailed CSV ---
        with open(damage_csv_out, 'w', newline='') as f:
            w = csv.writer(f)
            w.writerow(["Time (s)", "Accumulated Dose (rads)", "EQ (rads)", "Ratio", "Damage Level"])
            for t, dose, EQ, ratio, lvl in zip(time_steps, dose_arr, EQ_used_over_time, ratios_over_time, damage_levels_over_time):
                w.writerow([float(t), float(dose), float(EQ), float(ratio), int(lvl)])

        # --- Plot ---
        plt.figure(figsize=(8, 4))
        x_hours = time_steps / 3600.0
        plt.plot(x_hours, damage_levels_over_time, drawstyle='steps-post', linewidth=2.5, marker='o', markersize=4)
        plt.title(f"Damage Levels — {label} (Batch {batch_num})")
        plt.xlabel("Time (hours)")
        plt.ylabel("Damage Level (1–5)")
        plt.ylim(1, 6)
        plt.yticks([1, 2, 3, 4, 5])
        plt.grid(True)
        plt.tight_layout()
        plt.savefig(png_out, dpi=150)
        plt.close()

        print(f"[{label}] Final Level = {damage_levels_over_time[-1]} | Max Ratio = {ratios_over_time.max():.2f}")

        # ============================================================
        # ADDITION: COPY SPHERE DAMAGE LEVELS CSVs TO CURRENT FOLDER
        # ============================================================
        if label == "Sphere" and batch_num in [1, 4, 5, 7]:
            dest_filename = f"sphere_damage_levels_{batch_num}.csv"
            dest_path = os.path.join(os.getcwd(), dest_filename)
            try:
                import shutil
                shutil.copyfile(damage_csv_out, dest_path)
                print(f"→ Copied Sphere CSV for Batch {batch_num} to current directory as '{dest_filename}'")
            except Exception as e:
                print(f"[WARNING] Could not copy Sphere CSV for Batch {batch_num}: {e}")

    print(f"[Batch {batch_num}] Damage Levels generated! ✅")







=== Batch 1 — Damage Levels (time-resolved) ===
[Point Source] Final Level = 5 | Max Ratio = 20300.53
[With Shield] Final Level = 5 | Max Ratio = 794500.29
[Sphere] Final Level = 5 | Max Ratio = 356.50
→ Copied Sphere CSV for Batch 1 to current directory as 'sphere_damage_levels_1.csv'
[Circular Base] Final Level = 5 | Max Ratio = 63971.40
[Batch 1] Damage Levels generated! ✅

=== Batch 2 — Damage Levels (time-resolved) ===
[Point Source] Final Level = 5 | Max Ratio = 16039.55
[With Shield] Final Level = 5 | Max Ratio = 628956.71
[Sphere] Final Level = 5 | Max Ratio = 311.85
[Circular Base] Final Level = 5 | Max Ratio = 50631.38
[Batch 2] Damage Levels generated! ✅

=== Batch 3 — Damage Levels (time-resolved) ===
[Point Source] Final Level = 5 | Max Ratio = 5394.32
[With Shield] Final Level = 5 | Max Ratio = 210613.35
[Sphere] Final Level = 5 | Max Ratio = 37.83
[Circular Base] Final Level = 5 | Max Ratio = 16962.23
[Batch 3] Damage Levels generated! ✅

=== Batch 4 — Damage Levels (ti

In [4]:
import pandas as pd
import os
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import numpy as np


# ============================
# 1) Import the level CSV files
# ============================
df1 = pd.read_csv("sphere_damage_levels_1.csv")
df4 = pd.read_csv("sphere_damage_levels_4.csv")
df5 = pd.read_csv("sphere_damage_levels_5.csv")
df7 = pd.read_csv("sphere_damage_levels_7.csv")

# ===================================
# 2) Instrumentation to location definition
# ===================================
instrumentation_by_location = {
    "WETWELL": ["Suppression Pool Thermocouples"],
    "DRYWELL": [
        "SRV Position Indicators",
        "Radiation Detectors in Containment",
        "Boron Injection Pressure Transmitters",
        "Boron Tank Level Transmitter",
        "Drywell Pressure Transmitters",
        "Containment Pressure Transmitters",
        "Containment Thermocouples",
        "Vessel Level (Wide Range)",
        "Vessel Level (Fuel Range)",
        "Vessel Pressure Transmitters",
        "Feedwater Flow Transmitters",
        "Drywell Thermocouples",
        "Radiation Detectors in Drywell",
        "Control Rods Position",
    ],
    "LOWER_HEAD": ["Vessel Thermocouples (Lower Head)", "FW Thermocouples"],
    "DOME": ["Vessel Thermocouples (Upper Head)"],
    "RECIRCULATION_PUMPS": [
        "Suction Pipes Thermocouples",
        "Recirculation Flow Transmitters",
        "Recirculation Pressure Transmitters",
    ],
    "ANNULUS": ["Average Reactor Power Monitors"],
    "AUXILIARY_BUILDING": [
        "Containment Hydrogen Concentration Analyzer",
        "Drywell Hydrogen Concentration Analyzer",
        "Feedwater Pressure Transmitters",
        "HPCS Flow Transmitter",
        "Suppression Pool Level Transmitters",
    ],
    "FUEL_BUILDING": [],
}

# =================================
# 3) Link the location to level CSV files
# =================================
level_map = {
    "WETWELL": 4,
    "DRYWELL": 7,
    "LOWER_HEAD": 1,
    "DOME": 1,
    "RECIRCULATION_PUMPS": 1,
    "ANNULUS": 1,
    "AUXILIARY_BUILDING": 5,
    "FUEL_BUILDING": 5,
}

# ==============================
# 4) CSV to dict conversion
# ==============================
level_df_map = {
    1: df1.to_dict(orient="records"),
    4: df4.to_dict(orient="records"),
    5: df5.to_dict(orient="records"),
    7: df7.to_dict(orient="records"),
}

# ===================================================
# 5) Link instrument with damage level
# ===================================================
damage_by_instrument = {}
for location, instruments in instrumentation_by_location.items():
    level = level_map[location]
    damage_data = level_df_map[level]
    for instr in instruments:
        damage_by_instrument[instr] = damage_data

# =========================================================
# 6) Actions and Guides
# =========================================================
actions_library = {
    "G1_BREACH_IN_VESSEL": {
        "TEMPERATURE DRYWELL": [
            "Drywell Thermocouples",
            "Containment Thermocouples",
            "Suppression Pool Thermocouples"
        ],
        "PRESSURE DRYWELL": [
            "Drywell Pressure Transmitters",
            "Containment Pressure Transmitters"
        ],
        "RADIATION CONTAINMENT": [
            "Radiation Detectors in Containment",
            "Radiation Detectors in Drywell",
        ],
        "HYDROGEN CONTAINMENT": [
            "Containment Hydrogen Concentration Analyzer",
            "Drywell Hydrogen Concentration Analyzer"
        ]
    },
    "G2_BORON_INJECTION": {
        "LEVEL BORON TANK": [
            "Boron Tank Level Transmitter",
            "Boron Injection Pressure Transmitters"
        ],
        "INJECTED FLOW RPV": [
            "Feedwater Flow Transmitters",
            "HPCS Flow Transmitter",
            "Boron Injection Pressure Transmitters",
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)",
            "Feedwater Pressure Transmitters"
        ],
        "LEVEL RPV": [
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)"
        ],
        "PRESSURE RPV": [
            "Vessel Pressure Transmitters",
            "Recirculation Pressure Transmitters"
        ],
        "TEMPERATURE RPV": [
            "Vessel Thermocouples (Upper Head)",
            "Vessel Thermocouples (Lower Head)",
            "FW Thermocouples",
            "Suction Pipes Thermocouples"
        ]
    },
    "G3_SPRAY": {
        "PRESSURE CONTAINMENT": [
            "Containment Pressure Transmitters",
            "Drywell Pressure Transmitters"
        ],
        "LEVEL CONTAINMENT": [
            "Suppression Pool Level Transmitters",
            "Suppression Pool Thermocouples"
        ],
        "RADIATION CONTAINMENT": [
            "Radiation Detectors in Containment",
            "Radiation Detectors in Drywell"
        ],
        "HYDROGEN CONTAINMENT": [
            "Containment Hydrogen Concentration Analyzer",
            "Drywell Hydrogen Concentration Analyzer"
        ]
    },
    "G4_VENTING": {
        "PRESSURE CONTAINMENT": [
            "Containment Pressure Transmitters",
            "Drywell Pressure Transmitters"
        ],
        "LEVEL CONTAINMENT": [
            "Suppression Pool Level Transmitters",
            "Suppression Pool Thermocouples"
        ],
        "HYDROGEN CONTAINMENT": [
            "Containment Hydrogen Concentration Analyzer",
            "Drywell Hydrogen Concentration Analyzer"
        ],
        "INJECTED FLOW RPV": [
            "Feedwater Flow Transmitters",
            "HPCS Flow Transmitter",
            "Boron Injection Pressure Transmitters",
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)",
            "Feedwater Pressure Transmitters"
        ],
    },
    "G5_INJECTION_RPV": {
        "INJECTED FLOW RPV": [
            "Feedwater Flow Transmitters",
            "HPCS Flow Transmitter",
            "Boron Injection Pressure Transmitters",
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)",
            "Feedwater Pressure Transmitters"
        ],
        "LEVEL RPV": [
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)"
        ],
        "PRESSURE RPV": [
            "Vessel Pressure Transmitters",
            "Recirculation Pressure Transmitters"
        ],
        "TEMPERATURE RPV": [
            "Vessel Thermocouples (Upper Head)",
            "Vessel Thermocouples (Lower Head)",
            "FW Thermocouples",
            "Suction Pipes Thermocouples"
        ]
    },
    "G6_INJECTION_RPV": {
        "INJECTED FLOW RPV": [
            "Feedwater Flow Transmitters",
            "HPCS Flow Transmitter",
            "Boron Injection Pressure Transmitters",
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)",
            "Feedwater Pressure Transmitters"
        ],
        "LEVEL RPV": [
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)"
        ],
        "PRESSURE RPV": [
            "Vessel Pressure Transmitters",
            "Recirculation Pressure Transmitters"
        ],
        "TEMPERATURE RPV": [
            "Vessel Thermocouples (Upper Head)",
            "Vessel Thermocouples (Lower Head)",
            "FW Thermocouples",
            "Suction Pipes Thermocouples"
        ]
    },
    "G7_INJECTION_PC_UP_TO_TAF": {
        "LEVEL RPV": [
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)"
        ],
        "PRESSURE RPV": [
            "Vessel Pressure Transmitters",
            "Recirculation Pressure Transmitters"
        ],
        "LEVEL CONTAINMENT": [
            "Suppression Pool Level Transmitters",
            "Suppression Pool Thermocouples"
        ],
        "PRESSURE SP": [
            "Suppression Pool Thermocouples",
            "Containment Pressure Transmitters",
            "Drywell Pressure Transmitters"
        ]
    },
    "G9_SUPPORT_VENTING_VESSEL": {
        "PRESSURE RPV": [
            "Vessel Pressure Transmitters",
            "Recirculation Pressure Transmitters"
        ],
        "INJECTED FLOW RPV": [
            "Feedwater Flow Transmitters",
            "HPCS Flow Transmitter",
            "Boron Injection Pressure Transmitters",
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)",
            "Feedwater Pressure Transmitters"
        ],
        "RADIATION CONTAINMENT": [
            "Radiation Detectors in Containment",
            "Radiation Detectors in Drywell"
        ],
        "HYDROGEN CONTAINMENT": [
            "Containment Hydrogen Concentration Analyzer",
            "Drywell Hydrogen Concentration Analyzer"
        ]
    },
    "G10_EVIDENCES_LOCA": {
        "PRESSURE RPV": [
            "Vessel Pressure Transmitters",
            "Recirculation Pressure Transmitters"
        ],
        "LEVEL RPV": [
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)"
        ],
        "PRESSURE DRYWELL": [
            "Drywell Pressure Transmitters",
            "Containment Pressure Transmitters"
        ],
        "SUPPRESSION POOL TEMPERATURE": [
            "Suppression Pool Thermocouples",
            "Drywell Thermocouples",
            "Containment Thermocouples"
        ],
        "RADIATION CONTAINMENT": [
            "Radiation Detectors in Containment",
            "Radiation Detectors in Drywell"
        ],
        "HYDROGEN CONTAINMENT": [
            "Containment Hydrogen Concentration Analyzer",
            "Drywell Hydrogen Concentration Analyzer"
        ]
    },
    "G12_FLOOD_RPV_UP_TO_TAF": {
        "PRESSURE RPV": [
            "Vessel Pressure Transmitters",
            "Recirculation Pressure Transmitters"
        ],
        "LEVEL RPV": [
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)"
        ],
        "LEVEL CONTAINMENT": [
            "Suppression Pool Level Transmitters",
            "Suppression Pool Thermocouples"
        ],
        "SRV POSITION": ["SRV Position Indicators"],
        "RADIATION CONTAINMENT": [
            "Radiation Detectors in Containment",
            "Radiation Detectors in Drywell"
        ]
    },
    "G13_SUPPORT_VENTING_RPV": {
        "PRESSURE RPV": [
            "Vessel Pressure Transmitters",
            "Recirculation Pressure Transmitters"
        ],
        "INJECTED FLOW RPV": [
            "Feedwater Flow Transmitters",
            "HPCS Flow Transmitter",
            "Boron Injection Pressure Transmitters",
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)",
            "Feedwater Pressure Transmitters"
        ],
        "SRV POSITION": ["SRV Position Indicators"],
        "RADIATION CONTAINMENT": [
            "Radiation Detectors in Containment",
            "Radiation Detectors in Drywell"
        ],
        "HYDROGEN CONTAINMENT": [
            "Containment Hydrogen Concentration Analyzer",
            "Drywell Hydrogen Concentration Analyzer"
        ]
    },
    "G14_DETERMINE_INVESSEL_RETENTION": {
        "PRESSURE RPV": [
            "Vessel Pressure Transmitters",
            "Recirculation Pressure Transmitters"
        ],
        "LEVEL RPV": [
            "Vessel Level (Wide Range)",
            "Vessel Level (Fuel Range)"
        ],
        "TEMPERATURE RPV": [
            "Vessel Thermocouples (Upper Head)",
            "Vessel Thermocouples (Lower Head)",
            "FW Thermocouples",
            "Suction Pipes Thermocouples"
        ],
        "LEVEL CONTAINMENT": [
            "Suppression Pool Level Transmitters",
            "Suppression Pool Thermocouples"
        ]
    },
    "GP": {
        "LEVEL CONTAINMENT": [
            "Suppression Pool Level Transmitters",
            "Suppression Pool Thermocouples"
        ],
        "SRV POSITION": ["SRV Position Indicators"],
        "PRESSURE RPV": [
            "Vessel Pressure Transmitters",
            "Recirculation Pressure Transmitters"
        ],
        "PRESSURE DRYWELL": [
            "Drywell Pressure Transmitters",
            "Containment Pressure Transmitters"
        ]
    },
    "GQ": {
        "REACTOR POWER": [
            "Average Reactor Power Monitors",
            "Control Rods Position"
        ]
    },
    "GT/D": {
        "TEMPERATURE DRYWELL": [
            "Drywell Thermocouples",
            "Containment Thermocouples",
            "Suppression Pool Thermocouples"
        ]
    },
    "GT/PC": {
        "TEMPERATURE CONTAINMENT": [
            "Containment Thermocouples",
            "Drywell Thermocouples",
            "Suppression Pool Thermocouples"
        ]
    },
    "GT/SP": {
        "TEMPERATURE SUPPRESSION POOL": [
            "Suppression Pool Thermocouples",
            "Drywell Thermocouples",
            "Containment Thermocouples"
        ]
    },
    "GH/PC": {
        "PRESSURE CONTAINMENT": [
            "Containment Pressure Transmitters",
            "Containment Pressure Transmitter (Auxiliary Building)",
            "Drywell Pressure Transmitters",
            "Drywell Pressure Transmitter (Auxiliary Building)"
        ],
        "HYDROGEN CONTAINMENT": [
            "Containment Hydrogen Concentration Analyzer",
            "Drywell Hydrogen Concentration Analyzer"
        ]
    },
    "GR/PC": {
        "PRESSURE CONTAINMENT": [
            "Containment Pressure Transmitters",
            "Containment Pressure Transmitter (Auxiliary Building)",
            "Drywell Pressure Transmitters",
            "Drywell Pressure Transmitter (Auxiliary Building)"
        ],
        "RADIATION CONTAINMENT": [
            "Radiation Detectors in Containment",
            "Radiation Detectors in Drywell"
        ]
    },
    "GL/SP": {
        "LEVEL CONTAINMENT": [
            "Suppression Pool Level Transmitters",
            "Suppression Pool Thermocouples"
        ]
    }
}

actions_short = { k.split("_")[0]: v for k,v in actions_library.items() }


# SAMG1 and SAMG2
SAMG1 = {
    "RC_F1": ["G1", "G3", "G4", "G5", "G6", "G7"],
    "RC_F2": ["G12", "G3", "G4", "G7", "G9", "G13"],
    "RC_F3": ["G3", "G4", "G12"],
    "RC_F4": ["G14", "G3", "G4", "G6"],
    "RC_F5": ["G3", "G4", "G5", "G7", "G14"],
    "RC_Q": ["G1", "G2", "GQ"],
    "RC_P": ["GP"],
}

SAMG2 = {
    "DW_T": ["GT/D"],
    "CN_T": ["GT/PC"],
    "SP_T": ["GT/SP"],
    "PC_P": ["G4"],
    "PC_H": ["GH/PC"],
    "PC_R": ["GR/PC"],
    "SP_L": ["GL/SP"],
}

# =========================================================
# 7) Utilities functions
# =========================================================
def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)

def _detect_interval_key(sample_records):
    if not sample_records:
        return "Interval"
    keys = list(sample_records[0].keys())
    lowmap = {k.lower(): k for k in keys}
    for cand in ["intervalo", "interval"]:
        if cand in lowmap:
            return lowmap[cand]
    for k in keys:
        if "interval" in k.lower():
            return k
    return keys[0]

def _detect_level_key(sample_records):
    if not sample_records:
        return "Damage Level"
    keys = list(sample_records[0].keys())
    lowmap = {k.lower(): k for k in keys}
    for cand in ["nivel de daño", "nivel_de_daño", "damage level", "damage_level", "damage"]:
        if cand in lowmap:
            return lowmap[cand]
    for k in keys:
        kl = k.lower()
        if "damage" in kl or "daño" in kl:
            return k
    return keys[-1]

# === Coloca esto junto a tus utils, antes de generate_graphs ===
ACTIONS_R_DIR = "ACTIONS_CSV_R"
create_directory(ACTIONS_R_DIR)

# Conjunto global para evitar duplicados entre múltiples llamadas (SAMG1_R y SAMG2_R)
ACTIONS_SAVED = set()


# =========================================================
# 8) Functions to build dataframes and states
# =========================================================
def _levels_dataframe_for_measure(instrs, damage_by_instrument, fallback_df_records):
    sample = next((damage_by_instrument[i] for i in instrs if i in damage_by_instrument), fallback_df_records)
    interval_key = _detect_interval_key(sample)
    level_key = _detect_level_key(sample)
    intervals = [r[interval_key] for r in sample]
    data = {interval_key: intervals}
    for instr in instrs:
        recs = damage_by_instrument.get(instr)
        if not recs:
            data[instr] = [1] * len(intervals)
            continue
        vals_raw = [rec.get(level_key, None) for rec in recs]
        vals = [int(v) if v is not None and str(v).strip() != "" else 1 for v in vals_raw]
        if len(vals) < len(intervals):
            vals += [vals[-1] if vals else 1] * (len(intervals) - len(vals))
        elif len(vals) > len(intervals):
            vals = vals[:len(intervals)]
        data[instr] = vals
    df = pd.DataFrame(data)
    return interval_key, level_key, intervals, df

def _availability_dataframe_for_measure(instrs, damage_by_instrument, fallback_df_records):
    sample = next((damage_by_instrument[i] for i in instrs if i in damage_by_instrument), fallback_df_records)
    interval_key = _detect_interval_key(sample)
    level_key = _detect_level_key(sample)
    intervals = [r[interval_key] for r in sample]
    all_levels = []
    for instr in instrs:
        recs = damage_by_instrument.get(instr)
        if not recs:
            all_levels.append([1] * len(intervals))
        else:
            vals_raw = [rec.get(level_key, None) for rec in recs]
            vals = [int(v) if v is not None and str(v).strip() != "" else 1 for v in vals_raw]
            if len(vals) < len(intervals):
                vals += [vals[-1] if vals else 1] * (len(intervals) - len(vals))
            elif len(vals) > len(intervals):
                vals = vals[:len(intervals)]
            all_levels.append(vals)
    states, codes = [], []
    for i in range(len(intervals)):
        lvls_t = [levels[i] for levels in all_levels] if all_levels else []
        st = _classify_state_from_levels(lvls_t)
        states.append(st)
        codes.append(state_map[st])
    df = pd.DataFrame({interval_key: intervals, "state": states, "state_code": codes})
    return interval_key, intervals, df

# =========================================================
# 9) Plotting functions
# =========================================================
def plot_damage(instruments, damage_by_instrument, save_path, title):
    # Tomamos una muestra para detectar claves
    sample = next((damage_by_instrument[i] for i in instruments if i in damage_by_instrument), None)
    if not sample:
        return

    interval_key = _detect_interval_key(sample)   # aquí detectará "Time (s)"
    level_key    = _detect_level_key(sample)      # aquí detectará "Damage Level"

    fig, ax = plt.subplots(figsize=(14, 7), dpi=120)
    fig.patch.set_facecolor("#001F3F")
    ax.set_facecolor("#001F3F")

    # Paleta sencilla y sin marcadores (muchos puntos → carísimo)
    colors = plt.cm.tab10.colors

    for idx, instr in enumerate(instruments):
        recs = damage_by_instrument.get(instr)
        if not recs:
            continue

        # x = tiempo real; y = nivel de daño (1..5)
        x = np.asarray([float(r[interval_key]) for r in recs])
        y = np.asarray([int(r.get(level_key, 1)) for r in recs], dtype=int)

        # Decimado automático si hay demasiados puntos (p. ej. > 3000)
        max_points = 3000
        n = len(x)
        if n > max_points:
            step = int(np.ceil(n / max_points))
            x = x[::step]
            y = y[::step]

        ax.plot(
            x, y,
            linewidth=1.5,
            label=instr,
            color=colors[idx % len(colors)]
        )

    ax.set_xlabel("Time (s)", color="white", fontsize=14)
    ax.set_ylabel("Damage Level", color="white", fontsize=14)
    ax.set_yticks([1, 2, 3, 4, 5])
    ax.set_yticklabels([1, 2, 3, 4, 5], color="white", fontsize=12)
    ax.set_title(title, color="white", fontsize=20, pad=16)

    ax.tick_params(colors="white", labelsize=12)
    ax.grid(color="white", linestyle="--", alpha=0.2, linewidth=1)
    for spine in ax.spines.values():
        spine.set_edgecolor("white")

    # Leyenda compacta
    leg = ax.legend(loc="upper center", bbox_to_anchor=(0.5, -0.12),
                    ncol=min(len(instruments), 3), frameon=False, fontsize=11)
    for txt in (leg.get_texts() if leg else []):
        txt.set_color("white")

    plt.tight_layout()
    plt.savefig(save_path, facecolor=fig.get_facecolor(), dpi=150)
    plt.close(fig)


# =========================================================
# 10) Availability functions
# =========================================================
state_map = {"Good": 0, "Careful": 1, "Warning": 2, "Blind": 3}
zone_colors = {0: "#006400", 1: "#CCCC00", 2: "#FF8C00", 3: "#8B0000"}

def plot_availability(measure, intervals, states, save_path):
    # Convertimos estados a códigos 0..3
    values = np.asarray([state_map[s] for s in states], dtype=int)
    x = np.asarray(intervals, dtype=float)  # aquí será "Time (s)"

    # Decimado automático (mismo criterio)
    max_points = 3000
    n = len(x)
    if n > max_points:
        step = int(np.ceil(n / max_points))
        x = x[::step]
        values = values[::step]

    fig, ax = plt.subplots(figsize=(14, 6), dpi=120)
    fig.patch.set_facecolor("#001F3F")
    ax.set_facecolor("#001F3F")

    # Bandas de fondo por zona
    for lvl, col in zone_colors.items():
        ax.axhspan(lvl, lvl + 1, color=col, alpha=0.25)

    # Línea simple (sin marcadores)
    ax.plot(x, values, linewidth=2, color="#00CED1")

    ax.set_xlabel("Time (s)", color="white", fontsize=14)
    ax.set_ylabel("State", color="white", fontsize=14)
    ax.set_yticks(list(state_map.values()))
    ax.set_yticklabels(list(state_map.keys()), color="white", fontsize=12)
    ax.set_title(f"{measure} Availability", color="white", fontsize=20, pad=16)

    ax.tick_params(colors="white", labelsize=12)
    ax.grid(color="white", linestyle="--", alpha=0.2, linewidth=1)
    for spine in ax.spines.values():
        spine.set_edgecolor("white")

    plt.tight_layout()
    plt.savefig(save_path, facecolor=fig.get_facecolor(), dpi=150)
    plt.close(fig)


def _classify_state_from_levels(levels):
    if not levels:
        return "Blind"
    lvls = [int(x) for x in levels if x is not None]
    if all(l == 1 for l in lvls):
        return "Good"
    if all(l == 5 for l in lvls):
        return "Blind"
    if any(l in (3, 4) for l in lvls):
        return "Warning"
    if any(l == 2 for l in lvls) and all(l <= 2 for l in lvls):
        return "Careful"
    return "Warning"

# =========================================================
# 11) CSV and graphs generators
# =========================================================
def generate_graphs(samg_name, samg_dict, actions_map, damage_by_instrument):
    # (todo lo que ya hacía tu función)
    create_directory(samg_name)
    for subguide, codes in samg_dict.items():
        path_sub = os.path.join(samg_name, subguide)
        create_directory(path_sub)
        for code in codes:
            if code not in actions_map:
                continue
            path_code = os.path.join(path_sub, code)
            create_directory(path_code)
            for measure, instrs in actions_map[code].items():
                base = measure.replace(" ", "_").replace("/", "_")

                # Construir DF (igual que antes)
                interval_key, level_key, intervals, df_levels = _levels_dataframe_for_measure(
                    instrs, damage_by_instrument, df1.to_dict(orient="records")
                )

                # 1) Guardado ORIGINAL (igual)
                df_levels.to_csv(os.path.join(path_code, base + ".csv"), index=False, encoding="utf-8")

                # 2) Gráfico ORIGINAL (igual)
                plot_damage(instrs, damage_by_instrument, os.path.join(path_code, base + ".png"), measure)

                # 3) NUEVO: copia única por ACTION en carpeta global, con sufijo _R
                #    - No se repite aunque aparezca en varias G*/SAMG*
                if base not in ACTIONS_SAVED:
                    df_levels.to_csv(os.path.join(ACTIONS_R_DIR, base + "_R.csv"), index=False, encoding="utf-8")
                    ACTIONS_SAVED.add(base)



def generate_availability(samg_name, samg_dict, actions_map, damage_by_instrument):
    base = f"Availability_{samg_name}"
    create_directory(base)
    for subguide, codes in samg_dict.items():
        path_sub = os.path.join(base, subguide)
        create_directory(path_sub)
        for code in codes:
            if code not in actions_map:
                continue
            path_code = os.path.join(path_sub, code)
            create_directory(path_code)
            for measure, instrs in actions_map[code].items():
                safe = measure.replace(" ", "_").replace("/", "_")
                interval_key, intervals, df_av = _availability_dataframe_for_measure(
                    instrs, damage_by_instrument, df1.to_dict(orient="records")
                )
                df_av.to_csv(os.path.join(path_code, safe + "_availability.csv"), index=False, encoding="utf-8")
                plot_availability(measure, intervals, df_av["state"].tolist(), os.path.join(path_code, safe + "_availability.png"))

# =========================================================
# 12) Execution
# =========================================================
generate_graphs("SAMG1_R", SAMG1, actions_short, damage_by_instrument)
generate_graphs("SAMG2_R", SAMG2, actions_short, damage_by_instrument)
generate_availability("SAMG1_R", SAMG1, actions_short, damage_by_instrument)
generate_availability("SAMG2_R", SAMG2, actions_short, damage_by_instrument)


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pypost.codes.melcor import MELCOR

# =========================
# CARGA MELCOR
# =========================
file_path_melcor = "MC_Step.ptf"
file_index_melcor = MELCOR.openPlotFile(file_path_melcor)

def cargar(var_code: str):
    data = np.array(MELCOR.getData(file_index_melcor, var_code))
    if data.ndim != 2 or data.shape[1] < 2:
        raise RuntimeError(f"Unexpected format for {var_code}: shape={data.shape}")
    return data[:, 0], data[:, 1]

tiempo_s, TempVapor_Wetwell = cargar("CVH-TVAP_305")

def cargar_misma_malla(var_code: str):
    t, v = cargar(var_code)
    if not np.array_equal(tiempo_s, t):
        if np.allclose(tiempo_s, t, rtol=0, atol=1e-12):
            return v
        raise ValueError(f"La malla de tiempo en {var_code} no coincide con la referencia.")
    return v

Presion_Wetwell          = cargar_misma_malla("CVH-P_305")
TempVapor_Drywell        = cargar_misma_malla("CVH-TVAP_201")
Presion_Drywell          = cargar_misma_malla("CVH-P_201")
TempVapor_CabezaSuperior = cargar_misma_malla("CVH-TVAP_160")
Presion_CabezaSuperior   = cargar_misma_malla("CVH-P_160")
TempVapor_CabezaInferior = cargar_misma_malla("CVH-TVAP_120")
Presion_CabezaInferior   = cargar_misma_malla("CVH-P_120")
TempVapor_BombaRecir     = cargar_misma_malla("CVH-TVAP_106")
Presion_BombaRecir       = cargar_misma_malla("CVH-P_106")

# =========================
# CONSTANTES
# =========================
EQ_PRESSURE = 410000.0   # Pa
EQ_TEMP     = 422.0      # K
VESSEL_FAILURE = Presion_BombaRecir < 6_000_000.0  # bool array

variables_melcor = {
    "tiempo_s": tiempo_s,
    "TempVapor_Wetwell":        TempVapor_Wetwell,
    "Presion_Wetwell":          Presion_Wetwell,
    "TempVapor_Drywell":        TempVapor_Drywell,
    "Presion_Drywell":          Presion_Drywell,
    "TempVapor_CabezaSuperior": TempVapor_CabezaSuperior,
    "Presion_CabezaSuperior":   Presion_CabezaSuperior,
    "TempVapor_CabezaInferior": TempVapor_CabezaInferior,
    "Presion_CabezaInferior":   Presion_CabezaInferior,
    "TempVapor_BombaRecir":     TempVapor_BombaRecir,
    "Presion_BombaRecir":       Presion_BombaRecir,
    "EQ_PRESSURE":              EQ_PRESSURE,
    "EQ_TEMP":                  EQ_TEMP,
    "VESSEL_FAILURE":           VESSEL_FAILURE
}

# =========================
# UTILIDADES (histeresis, duración, constructores de niveles)
# =========================
def aplicar_histeresis(tiempo, niveles_condiciones, retardo_bajada_s):
    """
    Subida inmediata al nivel mayor; bajada solo tras 'retardo_bajada_s' continuos cumpliendo nivel menor.
    """
    out = np.zeros_like(niveles_condiciones)
    out[0] = niveles_condiciones[0]
    t_last = tiempo[0]
    for i in range(1, len(tiempo)):
        prev, want = out[i-1], niveles_condiciones[i]
        if want > prev:
            out[i] = want; t_last = tiempo[i]
        elif want < prev:
            out[i] = want if (tiempo[i] - t_last) >= retardo_bajada_s else prev
            if out[i] == want: t_last = tiempo[i]
        else:
            out[i] = prev
    return out

def duracion_continua(tiempo, mask_bool):
    dur = np.zeros_like(tiempo, dtype=float)
    for i in range(1, len(tiempo)):
        dur[i] = (dur[i-1] + (tiempo[i]-tiempo[i-1])) if (mask_bool[i] and mask_bool[i-1]) else (tiempo[i]-tiempo[i-1] if mask_bool[i] else 0.0)
    return dur

def niveles_base(n):
    return np.ones(n, dtype=int)

def set_or(mask_dest, mask_src, level):
    """Asigna 'level' donde mask_src True (sobrescribe niveles menores)."""
    mask_dest[mask_src] = level

def levels_cec(T, P, mults=((2,1.0),(3,1.5),(5,2.0)), eqT=EQ_TEMP, eqP=EQ_PRESSURE):
    lv = niveles_base(len(T))
    for L, m in mults:
        set_or(lv, (T > m*eqT) | (P > m*eqP), L)
    return lv

def levels_temp_steps(T, rules):
    """
    'rules' = lista ordenada de tuplas (level, cond_bool) que pisan niveles previos.
    Ej.: [(2, T>EQ), (3, T>673), (5, T>1530)]
    """
    lv = niveles_base(len(T))
    for L, cond in rules:
        set_or(lv, cond, L)
    return lv

def apply_and_save(name, lv_cond, delay_s):
    lv = aplicar_histeresis(tiempo_s, lv_cond, delay_s)
    guardar_nivel_instrumento(name, lv)
    return lv

# =========================
# CONTENEDOR NIVELES
# =========================
niveles_instrumentos = {}  # { nombre: ndarray[int] }

def guardar_nivel_instrumento(nombre: str, level_array: np.ndarray):
    if level_array.shape != tiempo_s.shape:
        raise ValueError(f"Longitud de niveles de '{nombre}' no coincide con tiempo.")
    niveles_instrumentos[nombre] = level_array.astype(int, copy=False)

# =========================
# INSTRUMENTOS (compactados por patrón)
# =========================

# 1) Suppression Pool Thermocouples (Twet/Pwet): 2:CEC>EQ, 3:T>673, 5:T>1530
T_wet, P_wet = TempVapor_Wetwell, Presion_Wetwell
lv = levels_temp_steps(T_wet, [(2, (T_wet>EQ_TEMP)|(P_wet>EQ_PRESSURE)),
                               (3, T_wet>673.0),
                               (5, T_wet>1530.0)])
apply_and_save("SuppressionPoolThermocouples", lv, 1800.0)

# 2) SRV Position Indicators: 2:CEC>EQ (Drywell), 5:Vessel Failure
T_dw, P_dw = TempVapor_Drywell, Presion_Drywell
lv = levels_temp_steps(T_dw, [(2, (T_dw>EQ_TEMP)|(P_dw>EQ_PRESSURE)),
                              (5, VESSEL_FAILURE)])
apply_and_save("SRVPositionIndicators", lv, 1800.0)

# 3) Radiation Detectors in Containment: CEC mults 1.0/1.5/2.0 (Drywell)
lv = levels_cec(T_dw, P_dw, mults=((2,1.0),(3,1.5),(5,2.0)))
apply_and_save("RadiationDetectorsContainment", lv, 1800.0)

# 4) Boron Injection Pressure Transmitters: CEC mults 1.0/1.5/2.0 (Drywell)
lv = levels_cec(T_dw, P_dw, mults=((2,1.0),(3,1.5),(5,2.0)))
apply_and_save("BoronInjectionPressureTransmitters", lv, 1800.0)

# 5) Boron Tank Level Transmitter:
#    2:CEC>EQ; 3: 30min T_dw>422 continuos; 5: T_dw>1530
dur_T422 = duracion_continua(tiempo_s, T_dw>422.0)
lv = levels_temp_steps(T_dw, [(2, (T_dw>EQ_TEMP)|(P_dw>EQ_PRESSURE)),
                              (3, dur_T422>=1800.0),
                              (5, T_dw>1530.0)])
apply_and_save("BoronTankLevelTransmitter", lv, 1800.0)

# 6) Drywell Pressure Transmitters: CEC mults 1.0/1.5/2.0
lv = levels_cec(T_dw, P_dw, mults=((2,1.0),(3,1.5),(5,2.0)))
apply_and_save("DrywellPressureTransmitters", lv, 1800.0)

# 7) Containment Pressure Transmitters: CEC mults 1.0/1.5/2.0
lv = levels_cec(T_dw, P_dw, mults=((2,1.0),(3,1.5),(5,2.0)))
apply_and_save("ContainmentPressureTransmitters", lv, 1800.0)

# 8) Containment Thermocouples (Drywell): 2:CEC>EQ; 3:T>673; 5:T>1530
lv = levels_temp_steps(T_dw, [(2, (T_dw>EQ_TEMP)|(P_dw>EQ_PRESSURE)),
                              (3, T_dw>673.0),
                              (5, T_dw>1530.0)])
apply_and_save("ContainmentThermocouples", lv, 1800.0)

# 9) Vessel Level (Wide Range): 2:CEC>EQ; 3:30min T_dw>422; 5:Vessel Failure
lv = levels_temp_steps(T_dw, [(2, (T_dw>EQ_TEMP)|(P_dw>EQ_PRESSURE)),
                              (3, dur_T422>=1800.0),
                              (5, VESSEL_FAILURE)])
apply_and_save("VesselLevelWideRange", lv, 1800.0)

# 10) Vessel Level (Fuel Range): igual que (9)
lv = levels_temp_steps(T_dw, [(2, (T_dw>EQ_TEMP)|(P_dw>EQ_PRESSURE)),
                              (3, dur_T422>=1800.0),
                              (5, VESSEL_FAILURE)])
apply_and_save("VesselLevelFuelRange", lv, 1800.0)

# 11) Vessel Pressure Transmitters:
#     2: (PRCS<0.25MPa & P_dw>0.2MPa) OR CEC>EQ ; 3:30min T_dw>422 ; 5:T_dw>1530
PRCS = Presion_BombaRecir
cond_A = (PRCS < 250_000.0) & (P_dw > 200_000.0)
lv = levels_temp_steps(T_dw, [(2, cond_A | ((T_dw>EQ_TEMP)|(P_dw>EQ_PRESSURE))),
                              (3, dur_T422>=1800.0),
                              (5, T_dw>1530.0)])
apply_and_save("VesselPressureTransmitters", lv, 1800.0)

# 12) Feedwater Flow Transmitters: CEC mults 1.0/1.5/2.0
lv = levels_cec(T_dw, P_dw, mults=((2,1.0),(3,1.5),(5,2.0)))
apply_and_save("FeedwaterFlowTransmitters", lv, 1800.0)

# 13) Drywell Thermocouples: 2:CEC>EQ; 3:T>673; 5:T>1530
lv = levels_temp_steps(T_dw, [(2, (T_dw>EQ_TEMP)|(P_dw>EQ_PRESSURE)),
                              (3, T_dw>673.0),
                              (5, T_dw>1530.0)])
apply_and_save("DrywellThermocouples", lv, 1800.0)

# 14) Radiation Detectors in Drywell: CEC mults 1.0/1.5/2.0
lv = levels_cec(T_dw, P_dw, mults=((2,1.0),(3,1.5),(5,2.0)))
apply_and_save("RadiationDetectorsDrywell", lv, 1800.0)

# 15) Control Rods Position: 2:CEC>EQ; 5:Vessel Failure
lv = levels_temp_steps(T_dw, [(2, (T_dw>EQ_TEMP)|(P_dw>EQ_PRESSURE)),
                              (5, VESSEL_FAILURE)])
apply_and_save("ControlRodsPosition", lv, 1800.0)

# 16) Recirculation Pressure Transmitters (P106/T106): 2/3/4 por umbrales; 5 por VF
P106, T106 = Presion_BombaRecir, TempVapor_BombaRecir
lv = niveles_base(len(tiempo_s))
for L, p_thr, t_thr in [(2, 8_000_000.0,  573.0),
                        (3,12_000_000.0,1100.0),
                        (4,17_200_000.0,1530.0)]:
    set_or(lv, (P106>p_thr)|(T106>t_thr), L)
set_or(lv, VESSEL_FAILURE, 5)
apply_and_save("RecirculationPressureTransmitters", lv, 10000.0)

# 17) Recirculation Flow Transmitters (P106/T106): idéntico a (16)
lv = niveles_base(len(tiempo_s))
for L, p_thr, t_thr in [(2, 8_000_000.0,  573.0),
                        (3,12_000_000.0,1100.0),
                        (4,17_200_000.0,1530.0)]:
    set_or(lv, (P106>p_thr)|(T106>t_thr), L)
set_or(lv, VESSEL_FAILURE, 5)
apply_and_save("RecirculationFlowTransmitters", lv, 10000.0)

# 18) Average Reactor Power Monitors: 2:CEC>EQ; 5:Vessel Failure
lv = levels_temp_steps(T_dw, [(2, (T_dw>EQ_TEMP)|(P_dw>EQ_PRESSURE)),
                              (5, VESSEL_FAILURE)])
apply_and_save("AverageReactorPowerMonitors", lv, 1800.0)

# 19) Vessel Thermocouples Upper Head (rangos)
T_uh = TempVapor_CabezaSuperior
lv = niveles_base(len(tiempo_s))
for L, cond in [(2, (T_uh>645.0) & (T_uh<873.0)),
                (3, (T_uh>923.0) & (T_uh<1173.0)),
                (4, (T_uh>1173.0)& (T_uh<1530.0)),
                (5,  T_uh>1570.0)]:
    set_or(lv, cond, L)
apply_and_save("VesselThermocouplesUpperHead", lv, 1800.0)

# 20) Vessel Thermocouples Lower Head (rangos; 5 por VF)
T_lh = TempVapor_CabezaInferior
lv = niveles_base(len(tiempo_s))
for L, cond in [(2, (T_lh>645.0) & (T_lh<873.0)),
                (3, (T_lh>923.0) & (T_lh<1173.0)),
                (4, (T_lh>1173.0)& (T_lh<1530.0))]:
    set_or(lv, cond, L)
set_or(lv, VESSEL_FAILURE, 5)
apply_and_save("VesselThermocouplesLowerHead", lv, 20000.0)

# 21) FW Thermocouples (Tcore = Cabeza Inferior): 2:[645,873), 4:[873,1530), 5:>1570
Tcore = TempVapor_CabezaInferior
lv = niveles_base(len(tiempo_s))
for L, cond in [(2, (Tcore>645.0) & (Tcore<873.0)),
                (4, (Tcore>873.0) & (Tcore<1530.0)),
                (5,  Tcore>1570.0)]:
    set_or(lv, cond, L)
apply_and_save("FWThermocouples", lv, 25000.0)

# 22) Suction Pipes Thermocouples (T106): 2/3/4 por rangos; 5:>1570
lv = niveles_base(len(tiempo_s))
for L, cond in [(2, (T106>645.0)  & (T106<873.0)),
                (3, (T106>923.0)  & (T106<1173.0)),
                (4, (T106>1173.0) & (T106<1530.0)),
                (5,  T106>1570.0)]:
    set_or(lv, cond, L)
apply_and_save("SuctionPipesThermocouples", lv, 100000.0)

# =========================
# EXPORTACIÓN (idéntica a la tuya)
# =========================
def exportar_niveles_y_graficas(niveles_instrumentos: dict, tiempo_s: np.ndarray,
                                carpeta_csv: str = "csv_niveles",
                                carpeta_figs: str = "graficas_niveles"):
    os.makedirs(carpeta_csv, exist_ok=True)
    os.makedirs(carpeta_figs, exist_ok=True)
    for nombre, niveles in niveles_instrumentos.items():
        if niveles.shape != tiempo_s.shape:
            print(f"[ADVERTENCIA] Longitud diferente en '{nombre}': tiempo={tiempo_s.shape}, niveles={niveles.shape}. Se omite.")
            continue
        safe = "".join(c if c.isalnum() or c in ("_", "-") else "_" for c in nombre)
        # CSV
        pd.DataFrame({"tiempo_s": tiempo_s, "nivel": niveles.astype(int)}).to_csv(
            os.path.join(carpeta_csv, f"{safe}_niveles.csv"), index=False
        )
        # Gráfica
        plt.figure()
        plt.step(tiempo_s, niveles, where="post")
        plt.xlabel("Tiempo [s]"); plt.ylabel("Nivel de daño (1–5)")
        plt.title(f"{nombre} – Nivel (1–5)")
        plt.yticks([1, 2, 3, 4, 5]); plt.ylim(0.8, 5.2); plt.grid(True); plt.tight_layout()
        ruta_png = os.path.join(carpeta_figs, f"{safe}_niveles.png")
        plt.savefig(ruta_png, dpi=200); plt.close()
        print(f"Guardado CSV: {os.path.join(carpeta_csv, f'{safe}_niveles.csv')}")
        print(f"Guardada gráfica: {ruta_png}")

exportar_niveles_y_graficas(niveles_instrumentos, tiempo_s)




In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ============================================================
# CONFIGURACIÓN
# ============================================================
carpeta = Path("csv_niveles")

state_order = {'Good':0, 'Careful':1, 'Warning':2, 'Blind':3}

ACTIONS_PyT_DIR = "ACTIONS_CSV_PyT"
os.makedirs(ACTIONS_PyT_DIR, exist_ok=True)
ACTIONS_PyT_SAVED = set()  # para evitar duplicados entre SAMG1_PyT y SAMG2_PyT

# ============================================================
# MAPEO CSV POR INSTRUMENTO
# ============================================================
instrument_to_csv = {
    "Average Reactor Power Monitors": "AverageReactorPowerMonitors",
    "Boron Injection Pressure Transmitters": "BoronInjectionPressureTransmitters",
    "Boron Tank Level Transmitter": "BoronTankLevelTransmitter",
    "Containment Pressure Transmitters": "ContainmentPressureTransmitters",
    "Containment Thermocouples": "ContainmentThermocouples",
    "Control Rods Position": "ControlRodsPosition",
    "Drywell Pressure Transmitters": "DrywellPressureTransmitters",
    "Drywell Thermocouples": "DrywellThermocouples",
    "Feedwater Flow Transmitters": "FeedwaterFlowTransmitters",
    "Feedwater Pressure Transmitters": "FeedwaterPressureTransmitters",
    "FW Thermocouples": "FWThermocouples",
    "Radiation Detectors in Containment": "RadiationDetectorsContainment",
    "Radiation Detectors in Drywell": "RadiationDetectorsDrywell",
    "Recirculation Flow Transmitters": "RecirculationFlowTransmitters",
    "Recirculation Pressure Transmitters": "RecirculationPressureTransmitters",
    "SRV Position Indicators": "SRVPositionIndicators",
    "Suction Pipes Thermocouples": "SuctionPipesThermocouples",
    "Suppression Pool Thermocouples": "SuppressionPoolThermocouples",  # base 'pp'
    "Suppression Pool Level Transmitters": "SuppressionPoolLevelTransmitters",
    "Vessel Level (Fuel Range)": "VesselLevelFuelRange",
    "Vessel Level (Wide Range)": "VesselLevelWideRange",
    "Vessel Pressure Transmitters": "VesselPressureTransmitters",
    "Vessel Thermocouples (Lower Head)": "VesselThermocouplesLowerHead",
    "Vessel Thermocouples (Upper Head)": "VesselThermocouplesUpperHead",
    "HPCS Flow Transmitter": "HPCSFlowTransmitter",
    "Containment Hydrogen Concentration Analyzer": "ContainmentHydrogenAnalyzer",
    "Drywell Hydrogen Concentration Analyzer": "DrywellHydrogenAnalyzer",
    # Opcionales si existen CSV:
    "Containment Pressure Transmitter (Auxiliary Building)": "ContainmentPressureTransmitterAux",
    "Drywell Pressure Transmitter (Auxiliary Building)": "DrywellPressureTransmitterAux",
}

# Alias (por si aparecen variantes de escritura)
name_aliases = {
    "Suppression Pool Thermocouples": "Supression Pool Thermocouples",
    "Suppression Pool Level Transmitters": "Suppresion Pool Level Transmitters",
}


# ============================================================
# UTILIDADES
# ============================================================
def _canon(s: str) -> str:
    return "".join(ch.lower() for ch in s if ch.isalnum())

def resolve_instrument_name(name: str, available: dict) -> str | None:
    name = name_aliases.get(name, name)
    if name in available:
        return name
    idx = { _canon(k): k for k in available.keys() }
    return idx.get(_canon(name))

def leer_csv_flexible(path: Path) -> pd.DataFrame:
    """Lee CSV con separador auto (, ; \t) y devuelve DF con columnas 'tiempo_s','nivel'."""
    seps = [",",";","\t","|"]
    last_err = None
    for sep in seps:
        try:
            df = pd.read_csv(path, sep=sep)
            if df.shape[1] < 1:
                continue
            # limpiar columnas unnamed
            df = df.loc[:, ~df.columns.astype(str).str.startswith("Unnamed")]
            cols = [c.lower().strip() for c in df.columns.astype(str)]

            # Caso NUEVO (ancho): columnas tiempo_s, nivel
            if "tiempo_s" in cols and "nivel" in cols:
                tcol = df.columns[cols.index("tiempo_s")]
                ncol = df.columns[cols.index("nivel")]
                out = df[[tcol,ncol]].rename(columns={tcol:"tiempo_s", ncol:"nivel"}).copy()
                out["tiempo_s"] = pd.to_numeric(out["tiempo_s"], errors="coerce")
                out["nivel"]    = pd.to_numeric(out["nivel"], errors="coerce")
                out = out.dropna(subset=["tiempo_s","nivel"])
                out["nivel"] = out["nivel"].round().clip(1,5).astype(int)
                out = out.sort_values("tiempo_s").reset_index(drop=True)
                return out

            # Caso ANTIGUO (largo): Intervalo / Nivel de Daño
            if "intervalo" in cols and "nivel de daño" in cols:
                icol = df.columns[cols.index("intervalo")]
                lcol = df.columns[cols.index("nivel de daño")]
                tmp = df[[icol,lcol]].rename(columns={icol:"Intervalo", lcol:"Nivel de Daño"}).copy()
                tmp["Intervalo"] = tmp["Intervalo"].astype(str)
                tmp["Nivel de Daño"] = pd.to_numeric(tmp["Nivel de Daño"], errors="coerce").fillna(1).astype(int).clip(1,5)
                # inventamos tiempo_s como 0,1,2,... (pasos)
                out = pd.DataFrame({
                    "tiempo_s": np.arange(len(tmp), dtype=float),
                    "nivel": tmp["Nivel de Daño"].to_numpy(int)
                })
                return out

            # Si tiene dos columnas cualesquiera, intentamos mapear a tiempo,nivel por posición
            if df.shape[1] >= 2:
                out = df.iloc[:, :2].copy()
                out.columns = ["tiempo_s","nivel"]
                out["tiempo_s"] = pd.to_numeric(out["tiempo_s"], errors="coerce")
                out["nivel"]    = pd.to_numeric(out["nivel"], errors="coerce")
                out = out.dropna(subset=["tiempo_s","nivel"])
                out["nivel"] = out["nivel"].round().clip(1,5).astype(int)
                out = out.sort_values("tiempo_s").reset_index(drop=True)
                return out

        except Exception as e:
            last_err = e
            continue
    # Si no pudimos leer, relanza último error
    raise last_err if last_err else RuntimeError(f"No se pudo leer {path}")

# ====== NUEVO: utilidades para construir y guardar CSV ======
def _levels_dataframe(instrs):
    """
    Devuelve (t_ref, df) donde df contiene 'tiempo_s' y,
    por cada instrumento, una columna con su nivel remuestreado (1..5).
    """
    t_ref = _reference_time(instrs)
    data = {"tiempo_s": t_ref}
    for i_name in instrs:
        resolved = resolve_instrument_name(i_name, damage_by_instrument) or i_name
        df = damage_by_instrument.get(resolved)
        y = _resample_to_reference(df, t_ref)
        data[resolved] = y.astype(int)
    out = pd.DataFrame(data)
    return t_ref, out

def _availability_dataframe(instrs):
    """
    Calcula availability y devuelve (t_ref, df) con columnas:
    'tiempo_s', 'state', 'state_code' (0..3).
    """
    t_ref = _reference_time(instrs)
    levels = []
    for i_name in instrs:
        resolved = resolve_instrument_name(i_name, damage_by_instrument) or i_name
        df = damage_by_instrument.get(resolved)
        y = _resample_to_reference(df, t_ref)
        levels.append(y.astype(int))

    states, codes = [], []
    for i in range(len(t_ref)):
        lvls_t = [int(arr[i]) for arr in levels]
        st = determine_state_from_levels(lvls_t)
        states.append(st)
        codes.append(state_order[st])

    df = pd.DataFrame({
        "tiempo_s": t_ref,
        "state": states,
        "state_code": np.array(codes, dtype=int)
    })
    return t_ref, df
# ============================================================


# ============================================================
# CARGA TODOS LOS INSTRUMENTOS + TIEMPO DE REFERENCIA
# ============================================================
raw_data = {}
ref_time = None

for visible_name, base in instrument_to_csv.items():
    ruta = carpeta / f"{base}_niveles.csv"
    if ruta.exists():
        df = leer_csv_flexible(ruta)
        raw_data[visible_name] = df
        if ref_time is None and not df.empty:
            ref_time = df["tiempo_s"].to_numpy()
    else:
        # se rellena luego con fallback
        pass

# Si no hay ningún CSV real, inventamos ref_time
if ref_time is None:
    ref_time = np.arange(100, dtype=float)  # 100 puntos por defecto

# Completar faltantes con nivel=1 y mismo eje temporal
for visible_name, base in instrument_to_csv.items():
    if visible_name not in raw_data:
        print(f"⚠ No encontrado: {base}_niveles.csv → se asigna nivel=1 con la malla temporal de referencia")
        raw_data[visible_name] = pd.DataFrame({
            "tiempo_s": ref_time,
            "nivel": np.ones_like(ref_time, dtype=int)
        })

# ============================================================
# PREPARACIÓN DE ESTRUCTURAS
# ============================================================
damage_by_instrument = {name: df for name, df in raw_data.items()}

# ============================================================
# LÓGICA AVAILABILITY (tu regla)
# ============================================================
def determine_state_from_levels(lvls: list[int]) -> str:
    if all(l == 1 for l in lvls):
        return 'Good'
    if any(l == 2 for l in lvls) and all(l <= 2 for l in lvls):
        return 'Careful'
    if all(l == 5 for l in lvls):
        return 'Blind'
    if any(l in (3,4) for l in lvls):
        return 'Warning'
    return 'Warning'

# ============================================================
# PLOTS
# ============================================================
def create_directory(path: str):
    if not os.path.exists(path):
        os.makedirs(path)

def _reference_time(instruments):
    """Devuelve el vector tiempo_s de referencia (el del primer instrumento con datos)."""
    for instr in instruments:
        resolved = resolve_instrument_name(instr, damage_by_instrument) or instr
        df = damage_by_instrument.get(resolved)
        if df is not None and not df.empty:
            return df["tiempo_s"].to_numpy()
    return ref_time

def _resample_to_reference(df: pd.DataFrame, t_ref: np.ndarray) -> np.ndarray:
    """
    Ajusta la serie (tiempo_s, nivel) al eje temporal t_ref:
    - si longitudes coinciden y tiempos iguales ≈: devuelve niveles tal cual
    - si difiere, interpola 'nearest' por escalón (usamos pandas merge_asof).
    """
    if df is None or df.empty:
        return np.ones_like(t_ref, dtype=int)
    src_t = df["tiempo_s"].to_numpy()
    src_y = df["nivel"].to_numpy(int)

    if len(src_t) == len(t_ref) and np.allclose(src_t, t_ref, rtol=0, atol=1e-9):
        return src_y

    # merge_asof para asignar al tiempo de referencia el nivel más cercano (look-up tipo nearest)
    a = pd.DataFrame({"t": t_ref})
    b = pd.DataFrame({"t": src_t, "y": src_y}).sort_values("t")
    out = pd.merge_asof(a.sort_values("t"), b, on="t", direction="nearest")
    y = out["y"].fillna(1).round().clip(1,5).astype(int).to_numpy()
    return y

def plot_damage(instruments, save_path, title):
    t_ref = _reference_time(instruments)
    fig, ax = plt.subplots(figsize=(16, 8), dpi=100)
    fig.patch.set_facecolor('#001F3F')
    ax.set_facecolor('#001F3F')

    markers = ['o','s','D','^','v','P','X','*','<','>']
    for idx, instr in enumerate(instruments):
        resolved = resolve_instrument_name(instr, damage_by_instrument) or instr
        df = damage_by_instrument.get(resolved)
        y = _resample_to_reference(df, t_ref)
        jitter = (idx - len(instruments)/2) * 0.05
        ax.plot(
            t_ref, y + jitter,
            marker=markers[idx % len(markers)],
            markersize=6,
            linewidth=1.8,
            label=resolved,
            markevery=max(1, len(t_ref)//20)
        )

    ax.set_xlabel("Time (s)", color='white', fontsize=20)
    ax.set_ylabel("Damage Level", color='white', fontsize=20)
    ax.set_yticks([1,2,3,4,5])
    ax.set_yticklabels([1,2,3,4,5], color='white', fontsize=18)
    ax.tick_params(colors='white', labelsize=16)
    for spine in ax.spines.values():
        spine.set_edgecolor('white')
    ax.grid(color='white', linestyle='--', alpha=0.2, linewidth=1)
    ax.set_title(title, color='white', fontsize=28, pad=24)

    # leyenda abajo
    leg_cols = len(instruments) if len(instruments) <= 2 else (len(instruments)+1)//2
    leg = ax.legend(
        loc='upper center', bbox_to_anchor=(0.5, -0.12),
        ncol=max(1, leg_cols), frameon=False, fontsize=14
    )
    for txt in leg.get_texts():
        txt.set_color('white')

    fig.subplots_adjust(bottom=0.25)
    plt.savefig(save_path, facecolor=fig.get_facecolor(), dpi=300)
    plt.close(fig)

def plot_availability(measure, t_ref, states, save_path):
    import numpy as np
    values = np.array([state_order[s] for s in states], dtype=int)

    fig, ax = plt.subplots(figsize=(16, 8), dpi=100)
    fig.patch.set_facecolor('#001F3F')
    ax.set_facecolor('#001F3F')

    # bandas por estado (Good/Careful/Warning/Blind)
    bands = {0:'#2E7D32', 1:'#F9A825', 2:'#EF6C00', 3:'#B71C1C'}
    for lvl, col in bands.items():
        ax.axhspan(lvl, lvl+1, color=col, alpha=0.25)

    # línea escalonada (mismo estilo que daño)
    ax.plot(t_ref, values, drawstyle='steps-post', marker='o',
            markersize=6, linewidth=1.8)

    # === Igualamos estilo de ejes al de "daño" ===
    ax.set_xlabel("Time (s)", color='white', fontsize=20)
    ax.set_ylabel("", color='white', fontsize=20)  # sin etiqueta textual
    ax.set_yticks([0,1,2,3])
    ax.set_yticklabels(list(state_order.keys()), color='white', fontsize=18)

    # ¡NO fijamos los xticks! (dejamos el auto-locator como en daño)
    ax.tick_params(colors='white', labelsize=16)
    for spine in ax.spines.values():
        spine.set_edgecolor('white')
    ax.grid(color='white', linestyle='--', alpha=0.2, linewidth=1)
    ax.set_title(f"{measure} Availability", color='white', fontsize=28, pad=24)

    fig.subplots_adjust(bottom=0.25)
    plt.savefig(save_path, facecolor=fig.get_facecolor(), dpi=300)
    plt.close(fig)


# ============================================================
# GENERACIÓN DE SALIDAS
# ============================================================
def create_directory(path: str):
    if not os.path.exists(path):
        os.makedirs(path)

def generate_graphs(samg_name, samg_dict, actions_map):
    create_directory(samg_name)
    for subguide, codes in samg_dict.items():
        path_sub = os.path.join(samg_name, subguide)
        create_directory(path_sub)
        for code in codes:
            if code not in actions_map:
                continue
            path_code = os.path.join(path_sub, code)
            create_directory(path_code)
            for measure, instrs in actions_map[code].items():
                base = measure.replace(' ', '_').replace('/', '_')

                # --- CSV de niveles por instrumento (remuestreados) ---
                _, df_levels = _levels_dataframe(instrs)
                out_csv = os.path.join(path_code, base + ".csv")
                df_levels.to_csv(out_csv, index=False, encoding="utf-8")

                # --- gráfico como antes ---
                plot_damage(instrs, os.path.join(path_code, base + ".png"), measure)

                # --- NUEVO: copia única por ACTION a carpeta global ACTIONS_CSV_PyT ---
                #     Mismo contenido del CSV anterior, nombre con sufijo _PyT.csv
                if base not in ACTIONS_PyT_SAVED:
                    df_levels.to_csv(
                        os.path.join(ACTIONS_PyT_DIR, base + "_PyT.csv"),
                        index=False, encoding="utf-8"
                    )
                    ACTIONS_PyT_SAVED.add(base)



def generate_availability(samg_name, samg_dict, actions_map):
    base_dir = f"Availability_{samg_name}"
    create_directory(base_dir)
    for subguide, codes in samg_dict.items():
        path_sub = os.path.join(base_dir, subguide)
        create_directory(path_sub)
        for code in codes:
            if code not in actions_map:
                continue
            path_code = os.path.join(path_sub, code)
            create_directory(path_code)
            for measure, instrs in actions_map[code].items():
                safe = measure.replace(' ', '_').replace('/', '_')
                # --- nuevo: CSV de availability ---
                t_ref, df_av = _availability_dataframe(instrs)
                df_av.to_csv(os.path.join(path_code, safe + "_availability.csv"),
                             index=False, encoding="utf-8")
                # --- gráfico como antes ---
                states = df_av["state"].tolist()
                plot_availability(measure, t_ref, states,
                                  os.path.join(path_code, safe + "_availability.png"))


# ============================================================
# EJECUCIÓN
# ============================================================
generate_graphs("SAMG1_PyT", SAMG1, actions_short)
generate_graphs("SAMG2_PyT", SAMG2, actions_short)
generate_availability("SAMG1_PyT", SAMG1, actions_short)
generate_availability("SAMG2_PyT", SAMG2, actions_short)


⚠ No encontrado: FeedwaterPressureTransmitters_niveles.csv → se asigna nivel=1 con la malla temporal de referencia
⚠ No encontrado: SuppressionPoolLevelTransmitters_niveles.csv → se asigna nivel=1 con la malla temporal de referencia
⚠ No encontrado: HPCSFlowTransmitter_niveles.csv → se asigna nivel=1 con la malla temporal de referencia
⚠ No encontrado: ContainmentHydrogenAnalyzer_niveles.csv → se asigna nivel=1 con la malla temporal de referencia
⚠ No encontrado: DrywellHydrogenAnalyzer_niveles.csv → se asigna nivel=1 con la malla temporal de referencia
⚠ No encontrado: ContainmentPressureTransmitterAux_niveles.csv → se asigna nivel=1 con la malla temporal de referencia
⚠ No encontrado: DrywellPressureTransmitterAux_niveles.csv → se asigna nivel=1 con la malla temporal de referencia


In [7]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ===========================
# CONFIGURACIÓN
# ===========================
VARIABLES = [
    "TEMPERATURE_DRYWELL",
    "PRESSURE_DRYWELL",
    "RADIATION_CONTAINMENT",
    "HYDROGEN_CONTAINMENT",
    "LEVEL_BORON_TANK",
    "INJECTED_FLOW_RPV",
    "LEVEL_RPV",
    "PRESSURE_RPV",
    "TEMPERATURE_RPV",
    "PRESSURE_CONTAINMENT",
    "LEVEL_CONTAINMENT",
    "PRESSURE_SP",
    "TEMPERATURE_SUPPRESSION_POOL",  # nombre canónico correcto
    "SRV_POSITION",
    "REACTOR_POWER",
    "TEMPERATURE_CONTAINMENT",
]

# Alias para encontrar archivos si en disco todavía aparece TEMPERATURE_SP
VAR_FILE_ALIASES = {
    "TEMPERATURE_SUPPRESSION_POOL": ["TEMPERATURE_SUPPRESSION_POOL", "TEMPERATURE_SP"]
}

DIR_R   = "ACTIONS_CSV_R"
DIR_PyT = "ACTIONS_CSV_PyT"

OUT_DIR_IMG = "GLOBAL_EVALUATION"        # imágenes y availability CSV
OUT_DIR_CSV = "GLOBAL_EVALUATION_CSV"    # CSV combinados (malla unida)
os.makedirs(OUT_DIR_IMG, exist_ok=True)
os.makedirs(OUT_DIR_CSV, exist_ok=True)

MARKERS = ["o","s","^","D","v","P","X","<",">","h"]
STATE_MAP   = {"Good":0, "Careful":1, "Warning":2, "Blind":3}
ZONE_COLORS = {0:"#006400", 1:"#CCCC00", 2:"#FF8C00", 3:"#8B0000"}

# ===========================
# UTILIDADES DE FICHEROS
# ===========================
def resolve_paths_for_var(varname: str):
    """Devuelve (ruta_R, ruta_PyT) probando alias si hace falta."""
    bases = VAR_FILE_ALIASES.get(varname, [varname])
    for base in bases:
        rp = os.path.join(DIR_R,   f"{base}_R.csv")
        pp = os.path.join(DIR_PyT, f"{base}_PyT.csv")
        if os.path.exists(rp) and os.path.exists(pp):
            return rp, pp
    # por si el usuario ya renombró todo: intentamos la canónica
    return os.path.join(DIR_R, f"{varname}_R.csv"), os.path.join(DIR_PyT, f"{varname}_PyT.csv")

def load_timeseries_csv(path: str) -> pd.DataFrame:
    """
    Lee un CSV de serie temporal y normaliza a:
      - 'tiempo_s' como columna de tiempo
      - columnas por instrumento, enteras 1..5
    """
    if not os.path.exists(path):
        raise FileNotFoundError(path)
    df = pd.read_csv(path)

    # Detecta la columna de tiempo (varias posibilidades)
    time_candidates = ["tiempo_s", "Time (s)", "time_s", "t", "time", "Time"]
    tcol = None
    for c in df.columns:
        if str(c).strip() in time_candidates:
            tcol = c; break
    if tcol is None:
        low = {str(c).strip().lower(): c for c in df.columns}
        for k in ["tiempo_s","time (s)","time_s","t","time"]:
            if k in low:
                tcol = low[k]; break
    if tcol is None:
        raise ValueError(f"{path}: no encuentro columna de tiempo.")

    df = df.rename(columns={tcol: "tiempo_s"}).copy()
    df["tiempo_s"] = pd.to_numeric(df["tiempo_s"], errors="coerce")
    df = df.dropna(subset=["tiempo_s"]).sort_values("tiempo_s").reset_index(drop=True)

    inst_cols = [c for c in df.columns if c != "tiempo_s"]
    for c in inst_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce").round().clip(1,5).astype("Int64")
    return df

# ===========================
# FUSIÓN R + PyT
# ===========================
def union_time_grid(df_r: pd.DataFrame | None, df_p: pd.DataFrame | None) -> np.ndarray:
    times = set()
    if df_r is not None: times.update(df_r["tiempo_s"].dropna().tolist())
    if df_p is not None: times.update(df_p["tiempo_s"].dropna().tolist())
    return np.array(sorted(times), dtype=float) if times else np.array([], dtype=float)

def ffill_at_times(df: pd.DataFrame | None, inst: str, t_union: np.ndarray) -> np.ndarray:
    """
    Para cada t_union, asigna el último valor <= t (carry-forward).
    Antes del primer dato: usa el primer valor. Si df no tiene la columna: NaN.
    """
    if df is None or inst not in df.columns:
        return np.full(len(t_union), np.nan, dtype=float)
    src = df[["tiempo_s", inst]].dropna(subset=["tiempo_s"]).sort_values("tiempo_s")
    a = pd.DataFrame({"t": t_union})
    b = src.rename(columns={"tiempo_s":"t"}).copy()
    out = pd.merge_asof(a, b, on="t", direction="backward")
    y = out[inst].to_numpy(dtype=float)

    # Antes del primer dato → primer valor disponible
    if len(src) > 0 and np.isnan(y).any():
        first_val = float(src[inst].iloc[0]) if not pd.isna(src[inst].iloc[0]) else np.nan
        y[np.isnan(y)] = first_val

    # Si todo NaN (columna corrupta), usa 1
    if np.all(np.isnan(y)):
        y = np.ones_like(y)
    return y

def compress_steps(x: np.ndarray, y: np.ndarray):
    """Devuelve puntos de cambio (step-post) y garantiza incluir el último x."""
    if len(x) == 0:
        return x, y
    x_comp = [x[0]]
    y_comp = [int(round(y[0]))]
    for i in range(1, len(x)):
        val = int(round(y[i]))
        if val != y_comp[-1]:
            x_comp.append(x[i]); y_comp.append(val)
    if x_comp[-1] != x[-1]:
        x_comp.append(x[-1]); y_comp.append(y_comp[-1])
    return np.array(x_comp), np.array(y_comp)

def build_combined_series(csv_r_path: str, csv_pyt_path: str):
    """
    Carga R y PyT (ambas series), alinea en malla unida, carry-forward en cada fuente,
    combina por máximo punto a punto. Devuelve:
      - t_union (np.ndarray)
      - df_combined (DataFrame): 'tiempo_s' + columnas por instrumento (1..5)
      - series_plot: {inst: (x_comp, y_comp)} comprimido para step-plot
      - insts (lista ordenada)
    """
    df_r = load_timeseries_csv(csv_r_path) if os.path.exists(csv_r_path) else None
    df_p = load_timeseries_csv(csv_pyt_path) if os.path.exists(csv_pyt_path) else None
    if df_r is None and df_p is None:
        raise FileNotFoundError(f"No encuentro ni {csv_r_path} ni {csv_pyt_path}")

    t_union = union_time_grid(df_r, df_p)
    if t_union.size == 0:
        return t_union, pd.DataFrame(), {}, []

    insts = set()
    if df_r is not None: insts.update([c for c in df_r.columns if c != "tiempo_s"])
    if df_p is not None: insts.update([c for c in df_p.columns if c != "tiempo_s"])
    insts = sorted(insts)

    data = {"tiempo_s": t_union}
    series_plot = {}
    for inst in insts:
        yr = ffill_at_times(df_r, inst, t_union)
        yp = ffill_at_times(df_p, inst, t_union)
        # máximo conservador; si ambos NaN → 1
        both_nan = np.isnan(yr) & np.isnan(yp)
        comb = np.where(both_nan, 1.0, np.nanmax(np.vstack([
            np.nan_to_num(yr, nan=-np.inf),
            np.nan_to_num(yp, nan=-np.inf)
        ]), axis=0))
        comb = np.round(comb).clip(1,5).astype(int)
        data[inst] = comb
        x_c, y_c = compress_steps(t_union, comb)
        series_plot[inst] = (x_c, y_c)

    df_combined = pd.DataFrame(data)
    return t_union, df_combined, series_plot, insts

# ===========================
# PLOTS
# ===========================
def auto_offsets(n: int, base=0.06):
    if n <= 1: return [0.0]
    idx = np.arange(n) - (n-1)/2.0
    return (idx / max(1, (n-1)/2.0)) * base

def plot_variable(name: str, series_plot: dict, insts: list):
    if not series_plot:
        return
    xmins = [s[0][0] for s in series_plot.values() if len(s[0])>0]
    xmaxs = [s[0][-1] for s in series_plot.values() if len(s[0])>0]
    if not xmins or not xmaxs:
        return
    xmin, xmax = min(xmins), max(xmaxs)

    fig, ax = plt.subplots(figsize=(12, 6))
    fig.patch.set_facecolor("#06243A")
    ax.set_facecolor("#06243A")

    offs = auto_offsets(len(insts), base=0.06)
    markers = {insts[i]: MARKERS[i % len(MARKERS)] for i in range(len(insts))}
    line_kwargs = dict(where="post", linewidth=2.0, markersize=7,
                       markeredgewidth=1.8, solid_capstyle="butt",
                       mfc="none", markevery=None)
    y_all = []
    for i, inst in enumerate(insts):
        x, y = series_plot[inst]
        if len(x) == 0: continue
        y_shift = y + offs[i]
        y_all.extend(list(y_shift))
        ln = ax.step(x, y_shift, marker=markers[inst], label=inst, **line_kwargs)[0]
        ln.set_zorder(3)

    ax.set_title(name.replace("_", " "), fontsize=34, fontweight="bold", color="white", pad=20)
    ax.set_xlabel("Time (s)", fontsize=20, color="white", labelpad=15)
    ax.set_ylabel("Damage Level", fontsize=24, color="white", labelpad=15)
    ax.set_xlim(xmin, xmax)
    if y_all:
        ymin = max(0.9, np.nanmin(y_all) - 0.1)
        ymax = min(5.1, np.nanmax(y_all) + 0.1)
        ax.set_ylim(ymin, ymax)
    ax.tick_params(axis="both", colors="white", labelsize=14)
    ax.grid(True, linestyle="--", alpha=0.35)
    for spine in ax.spines.values():
        spine.set_color("white"); spine.set_alpha(0.6)
    plt.subplots_adjust(bottom=0.24)
    ncols = min(4, max(2, int(math.ceil(len(insts)/2))))
    leg = ax.legend(loc="upper center", bbox_to_anchor=(0.5, -0.18),
                    ncol=ncols, frameon=False, fontsize=14,
                    handlelength=2.6, handletextpad=0.8, columnspacing=1.8)
    for txt in leg.get_texts():
        txt.set_color("white")

    out_path = os.path.join(OUT_DIR_IMG, f"{name}_COMBINED.png")
    plt.savefig(out_path, dpi=220, bbox_inches="tight", facecolor=fig.get_facecolor())
    plt.close(fig)
    print(f"✓ Guardado: {out_path}")

# ===========================
# AVAILABILITY
# ===========================
def classify_state_from_levels(levels):
    if not levels:
        return "Blind"
    lvls = [int(x) for x in levels if x is not None]
    if all(l == 1 for l in lvls): return "Good"
    if all(l == 5 for l in lvls): return "Blind"
    if any(l in (3,4) for l in lvls): return "Warning"
    if any(l == 2 for l in lvls) and all(l <= 2 for l in lvls): return "Careful"
    return "Warning"

def build_availability_from_combined(df_combined: pd.DataFrame) -> pd.DataFrame:
    """Calcula availability sobre la malla completa combinada."""
    times = df_combined["tiempo_s"].to_numpy()
    inst_cols = [c for c in df_combined.columns if c != "tiempo_s"]
    states, codes = [], []
    for i in range(len(times)):
        lvls = [int(df_combined.iloc[i][c]) for c in inst_cols]
        st = classify_state_from_levels(lvls)
        states.append(st)
        codes.append(STATE_MAP[st])
    return pd.DataFrame({"time_s": times, "state": states, "state_code": codes})

def plot_availability_timeseries(var_name: str, df_av: pd.DataFrame):
    times = df_av["time_s"].tolist()
    values = df_av["state_code"].tolist()
    fig, ax = plt.subplots(figsize=(16, 9), dpi=120)
    fig.patch.set_facecolor("#06243A")
    ax.set_facecolor("#06243A")
    for lvl, col in ZONE_COLORS.items():
        ax.axhspan(lvl, lvl + 1, color=col, alpha=0.28)
    ax.step(times, values, where="post", marker="o", mfc="none",
            markersize=6, linewidth=1.8, color="#00CED1")
    ax.set_xlim(min(times), max(times))
    ax.set_xlabel("Time (s)", fontsize=20, color="white", labelpad=15)
    ax.set_ylabel("Availability State", fontsize=20, color="white", labelpad=15)
    ax.set_yticks(list(STATE_MAP.values()))
    ax.set_yticklabels(list(STATE_MAP.keys()), color="white", fontsize=16)
    ax.set_title(f"{var_name.replace('_',' ')} Availability", color="white", fontsize=28, pad=40)
    ax.tick_params(colors="white", labelsize=14)
    ax.grid(color="white", linestyle="--", alpha=0.25, linewidth=1)
    for spine in ax.spines.values():
        spine.set_edgecolor("white")
    fig.subplots_adjust(top=0.88, bottom=0.18)
    out_png = os.path.join(OUT_DIR_IMG, f"{var_name}_availability.png")
    plt.savefig(out_png, facecolor=fig.get_facecolor(), dpi=220)
    plt.close(fig)
    print(f"✓ Guardado: {out_png}")

# ===========================
# EJECUCIÓN
# ===========================
for var in VARIABLES:
    # Corrige alias en disco: si existe TEMPERATURE_SP úsalo, pero el nombre lógico es TEMPERATURE_SUPPRESSION_POOL
    r_path, p_path = resolve_paths_for_var(var)

    if not (os.path.exists(r_path) and os.path.exists(p_path)):
        print(f"⚠️  Omito {var}: faltan {r_path} o {p_path}")
        continue

    # 1) Combina R+PyT
    t_union, df_combined, series_plot, insts = build_combined_series(r_path, p_path)
    if df_combined.empty or not insts:
        print(f"⚠️  Omito {var}: no hay datos tras combinar.")
        continue

    # 2) Guarda CSV combinado completo (malla unida)
    out_csv = os.path.join(OUT_DIR_CSV, f"{var}_combined.csv")
    df_combined.to_csv(out_csv, index=False)
    print(f"✓ Guardado: {out_csv}")

    # 3) Gráfica de niveles combinados por instrumento (step-post comprimido + offsets visuales)
    plot_variable(var, series_plot, insts)

    # 4) Availability sobre la malla completa
    df_av = build_availability_from_combined(df_combined)
    out_av_csv = os.path.join(OUT_DIR_IMG, f"{var}_availability.csv")
    df_av.to_csv(out_av_csv, index=False)
    print(f"✓ Guardado: {out_av_csv}")
    plot_availability_timeseries(var, df_av)



✓ Guardado: GLOBAL_EVALUATION_CSV\TEMPERATURE_DRYWELL_combined.csv
✓ Guardado: GLOBAL_EVALUATION\TEMPERATURE_DRYWELL_COMBINED.png
✓ Guardado: GLOBAL_EVALUATION\TEMPERATURE_DRYWELL_availability.csv
✓ Guardado: GLOBAL_EVALUATION\TEMPERATURE_DRYWELL_availability.png
✓ Guardado: GLOBAL_EVALUATION_CSV\PRESSURE_DRYWELL_combined.csv
✓ Guardado: GLOBAL_EVALUATION\PRESSURE_DRYWELL_COMBINED.png
✓ Guardado: GLOBAL_EVALUATION\PRESSURE_DRYWELL_availability.csv
✓ Guardado: GLOBAL_EVALUATION\PRESSURE_DRYWELL_availability.png
✓ Guardado: GLOBAL_EVALUATION_CSV\RADIATION_CONTAINMENT_combined.csv
✓ Guardado: GLOBAL_EVALUATION\RADIATION_CONTAINMENT_COMBINED.png
✓ Guardado: GLOBAL_EVALUATION\RADIATION_CONTAINMENT_availability.csv
✓ Guardado: GLOBAL_EVALUATION\RADIATION_CONTAINMENT_availability.png
✓ Guardado: GLOBAL_EVALUATION_CSV\HYDROGEN_CONTAINMENT_combined.csv
✓ Guardado: GLOBAL_EVALUATION\HYDROGEN_CONTAINMENT_COMBINED.png
✓ Guardado: GLOBAL_EVALUATION\HYDROGEN_CONTAINMENT_availability.csv
✓ Guardado: 